In [1]:
import sys; sys.path.append('../data_processing/')
import pandas as pd
from parse_dep import *
import re
from ast import literal_eval
import tokenizations
from datasets import load_dataset
pipeline = get_spacy_pipeline('bert')

2023-09-10 02:48:19.571745: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ThirdParty-8/platforms/linux64Gcc/gperftools-svn/lib:/opt/openfoam8/platforms/linux64GccDPInt32Opt/lib/openmpi-system:/opt/ThirdParty-8/platforms/linux64GccDPInt32/lib/openmpi-system:/usr/lib/x86_64-linux-gnu/openmpi/lib:/home/dsinghvi/OpenFOAM/dsinghvi-8/platforms/linux64GccDPInt32Opt/lib:/opt/site/8/platforms/linux64GccDPInt32Opt/lib:/opt/openfoam8/platforms/linux64GccDPInt32Opt/lib:/opt/ThirdParty-8/platforms/linux64GccDPInt32/lib:/opt/openfoam8/platforms/linux64GccDPInt32Opt/lib/dummy
2023-09-10 02:48:19.571774: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('bert.pred.mwe', sep='\t', names=[0, 'sentence', 'd'])
# df

In [3]:
def strip_g(l: list):
    return [i.replace("Ġ", "") for i in l]

def list_to_index_dict(input_list):                                                                                                               
    index_dict = {}

    for index, sublist in enumerate(input_list):
        for item in sublist:
            if item in index_dict:
                index_dict[item].append(index)
            else:
                index_dict[item] = [index]

    return index_dict

def map_mwes_together(x, mwe_type):
    """
    ~: for strong
    _: for weak
    """
    assert mwe_type in ["~", "_"], "MWE Type must be one of [~, _]"
    mapped_mwes = []
    for j, mwe in enumerate(x[mwe_type]):
        mapped_mwes.append([])
        for index in mwe:
            
            if index-1 in x['token_map_dict']:
                for val in x['token_map_dict'][index-1]:
                    mapped_mwes[j].append(val+1)
                
                
    return mapped_mwes



df = pd.DataFrame([["His career was halted in 1997 when he was diagnosed with Non @-@ Hodgkin_lymphoma .", {"toks": [["His", "PRON"], ["career", "NOUN"],        ["was", "AUX"], ["halted", "VERB"], ["in", "ADP"], ["1997", "NUM"], ["when", "SCONJ"], ["he", "PRON"], ["was", "AUX"], ["diagnosed", "VERB"],     ["with", "ADP"], ["Non", "PROPN"], ["@-@", "PROPN"], ["Hodgkin", "PROPN"], ["lymphoma", "NOUN"], [".", "PUNCT"]], "tags": ["O", "O", "O", "O-     social", "O", "O", "O", "O", "O", "O-cognition", "O", "O", "O", "B", "\u012a", "O"], "_": [[14, 15]], "~": [], "lemmas": ["his", "career", "wa",  "halt", "in", "1997", "when", "he", "wa", "diagnose", "with", "non", "@-@", "hodgkin", "lymphoma", "."]}]], columns = ['sentence', 'd'])
df = pd.concat([df, pd.json_normalize(df.d)],axis=1)
df['tokens'] = df['toks'].apply(lambda x: [y[0] for y in x])
df['sent'] = df['sentence'].apply(lambda x: x.replace("_", " ").replace("~"," "))

df['tokens_to_map'] = df.apply(lambda x: list(map(str, list(pipeline(x["sent"])))), axis=1)
df['token_map'] = df.apply(lambda x: tokenizations.get_alignments(x['tokens'],
                                                              strip_g(x['tokens_to_map']))[1], axis=1)
df['token_map_dict'] = df['token_map'].apply(lambda x: list_to_index_dict(x))
df['weak_mwe'] = df.apply(lambda x: map_mwes_together(x, "_"), axis=1)
df['strong_mwe'] = df.apply(lambda x: map_mwes_together(x, "~"), axis=1)

display(df)
df['token_map_dict'].iloc[0]

,sentence,d,toks,tags,_,~,lemmas,tokens,sent,tokens_to_map,token_map,token_map_dict,weak_mwe,strong_mwe
0,His career was halted in 1997 when he was diag...,"{'toks': [['His', 'PRON'], ['career', 'NOUN'],...","[[His, PRON], [career, NOUN], [was, AUX], [hal...","[O, O, O, O- social, O, O, O, O, O, O-cogn...","[[14, 15]]",[],"[his, career, wa, halt, in, 1997, when, he, wa...","[His, career, was, halted, in, 1997, when, he,...",His career was halted in 1997 when he was diag...,"[his, career, was, halted, in, 1997, when, he,...","[[0], [1], [2], [3], [4], [5], [6], [7], [8], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5...","[[16, 17, 18, 19, 20, 21, 22, 23]]",[]


{0: [0],
 1: [1],
 2: [2],
 3: [3],
 4: [4],
 5: [5],
 6: [6],
 7: [7],
 8: [8],
 9: [9],
 10: [10],
 11: [11],
 12: [12, 13, 14],
 13: [15, 16, 17, 18],
 14: [19, 20, 21, 22],
 15: [23]}

In [ ]:
from transformers import BertForMaskedLM, BertTokenizerFast

sent = "His career was halted in 1997 when he was diagnosed with Non @-@ Hodgkin_lymphoma ."

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', use_fast=True)
abc = tokenizer.tokenize(sent, padding=False,  truncation=True, max_length=50, return_tensors ='pt')
print(len(abc))
print(abc)
abc = tokenizer(sent, padding=False,  truncation=True, max_length=50, return_tensors ='pt').input_ids[0]
print(abc)
print(tokenizer.decode(abc))
print(len(abc))




tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
abc = tokenizer.tokenize(sent)
print(abc)
print(len(abc))

25
['his', 'career', 'was', 'halted', 'in', '1997', 'when', 'he', 'was', 'diagnosed', 'with', 'non', '@', '-', '@', 'ho', '##d', '##g', '##kin', '_', 'l', '##ym', '##ph', '##oma', '.']
tensor([  101,  2010,  2476,  2001, 12705,  1999,  2722,  2043,  2002,  2001,
        11441,  2007,  2512,  1030,  1011,  1030,  7570,  2094,  2290,  4939,
         1035,  1048, 24335,  8458,  9626,  1012,   102])


ModuleNotFoundError: No module named 'google.protobuf'